In [1]:
!pip install workalendar

In [2]:
import pandas as pd
from workalendar.america import Mexico
from datetime import datetime, timedelta

In [3]:

cal = Mexico()
start_date = datetime(2024, 1, 1)
end_date = datetime(2024, 12, 31)
dates = pd.date_range(start_date, end_date)

# Crear la tabla de calendario
is_working_day = [
    cal.is_working_day(date) and date.weekday() != 6
    for date in dates
]
calendar_df = pd.DataFrame({
    'Fecha': dates,
    'Es_dia_laboral': is_working_day
})
calendar_df['Mes'] = calendar_df['Fecha'].dt.strftime('%B')
meses_traduccion = {
    'January': 'Enero', 'February': 'Febrero', 'March': 'Marzo',
    'April': 'Abril', 'May': 'Mayo', 'June': 'Junio',
    'July': 'Julio', 'August': 'Agosto', 'September': 'Septiembre',
    'October': 'Octubre', 'November': 'Noviembre', 'December': 'Diciembre'
}
calendar_df['Mes'] = calendar_df['Mes'].map(meses_traduccion)

semana_santa = cal.holidays(2024)
semana_santa = [(date, name) for date, name in semana_santa if "Jueves Santo" in name or "Viernes Santo" in name]
for date, _ in semana_santa:
    calendar_df.loc[calendar_df['Fecha'] == date, 'Es_dia_laboral'] = False


route=r'c:\Users\MERCA\Desktop\Presupuesto_mensual\Presupuesto_mensual.xlsx'
establecimientos_df = pd.read_excel(route)
establecimientos_df['Mes'] = establecimientos_df['Atributo'].str.capitalize()
merged_df = calendar_df.merge(establecimientos_df[['Super/abarrotes', 'Mes','Columna1','Valor']], on='Mes', how='left')
merged_df['Dia_descontado'] = False
for i, row in merged_df.iterrows():
    if row['Super/abarrotes'] == 'Abarrotes':
        if not row['Es_dia_laboral']:
            merged_df.at[i, 'Dia_descontado'] = True
    elif row['Super/abarrotes'] == 'super':
        if row['Fecha'] in [date for date, _ in semana_santa] or not cal.is_working_day(row['Fecha']):
            merged_df.at[i, 'Dia_descontado'] = True
dias_laborales_filtrados = merged_df[~merged_df['Dia_descontado']]

print(dias_laborales_filtrados)

          Fecha  Es_dia_laboral        Mes Super/abarrotes  \
20   2024-01-02            True      Enero           super   
21   2024-01-02            True      Enero           super   
22   2024-01-02            True      Enero           super   
23   2024-01-02            True      Enero       Abarrotes   
24   2024-01-02            True      Enero           super   
...         ...             ...        ...             ...   
7448 2024-12-30            True  Diciembre       Abarrotes   
7449 2024-12-30            True  Diciembre       Abarrotes   
7450 2024-12-30            True  Diciembre       Abarrotes   
7451 2024-12-30            True  Diciembre       Abarrotes   
7452 2024-12-30            True  Diciembre       Abarrotes   

                Columna1        Valor  Dia_descontado  
20              Carrillo   5965450.37           False  
21             Guadalupe   4274451.04           False  
22     Lago de Patzcuaro   5593621.12           False  
23             Ecommerce   1085

In [4]:
dias_trabajados=dias_laborales_filtrados.groupby(['Mes','Columna1']).count()
dias_trabajados_presupuesto=dias_laborales_filtrados.merge(dias_trabajados, on=['Mes'])
dias_trabajados_presupuesto['presupuesto_diario']=dias_trabajados_presupuesto['Valor_x']/dias_trabajados_presupuesto['Valor_y']
dias_trabajados_presupuesto

,Fecha_x,Es_dia_laboral_x,Mes,Super/abarrotes_x,Columna1,Valor_x,Dia_descontado_x,Fecha_y,Es_dia_laboral_y,Super/abarrotes_y,Valor_y,Dia_descontado_y,presupuesto_diario
0,2024-01-02,True,Enero,super,Carrillo,5965450.37,False,22,22,22,22,22,271156.8350
1,2024-01-02,True,Enero,super,Carrillo,5965450.37,False,22,22,22,22,22,271156.8350
2,2024-01-02,True,Enero,super,Carrillo,5965450.37,False,22,22,22,22,22,271156.8350
3,2024-01-02,True,Enero,super,Carrillo,5965450.37,False,22,22,22,22,22,271156.8350
4,2024-01-02,True,Enero,super,Carrillo,5965450.37,False,22,22,22,22,22,271156.8350
...,...,...,...,...,...,...,...,...,...,...,...,...,...
106064,2024-12-30,True,Diciembre,Abarrotes,Ario de Rosales,1089913.53,False,20,20,20,20,20,54495.6765
106065,2024-12-30,True,Diciembre,Abarrotes,Ario de Rosales,1089913.53,False,20,20,20,20,20,54495.6765
106066,2024-12-30,True,Diciembre,Abarrotes,Ario de Rosales,1089913.53,False,20,20,20,20,20,54495.6765
106067,2024-12-30,True,Diciembre,Abarrotes,Ario de Rosales,1089913.53,False,20,20,20,20,20,54495.6765


In [8]:
df_limpio=[]
df_limpio=pd.DataFrame()
df_limpio['Fecha']=dias_trabajados_presupuesto['Fecha_x']
df_limpio['Sucursal']=dias_trabajados_presupuesto['Columna1']
df_limpio['Super/abarrotes']=dias_trabajados_presupuesto['Super/abarrotes_x']
df_limpio['Presupuesto']=dias_trabajados_presupuesto['presupuesto_diario']

In [11]:
df_limpio = df_limpio.groupby(['Fecha', 'Sucursal']).first().reset_index()
df_limpio

,Fecha,Sucursal,Super/abarrotes,Presupuesto
0,2024-01-02,Block 1,Abarrotes,559689.665909
1,2024-01-02,Block 2,Abarrotes,410321.840455
2,2024-01-02,Block 6,Abarrotes,185611.334545
3,2024-01-02,Block 8,Abarrotes,261367.800909
4,2024-01-02,Carrillo,super,271156.835000
...,...,...,...,...
5184,2024-12-30,Villas del Oriente,super,161134.231000
5185,2024-12-30,Villas del Pedregal,super,319283.153000
5186,2024-12-30,abastos matriz,Abarrotes,752879.379500
5187,2024-12-30,avios abastos,Abarrotes,54202.061000


In [13]:
with pd.ExcelWriter('Presupuesto_JC.xlsx', engine='openpyxl') as writer:
    df_limpio.to_excel(writer, sheet_name='Presupuesto_diario', index=False)
